# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


#### Creating list of filepaths to process original event csv data files

In [2]:

def get_files(file_path:str):
    """get all csv files from file_path  """
    all_files = []
    for root, dirs, files in os.walk(file_path):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

In [3]:
#List of all csv files
all_files = get_files('./event_data')

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in all_files:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in all_files:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))

In [6]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [30]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [31]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)
                    

    




#### Set Keyspace

In [32]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [33]:
## query1:
## The following table is designed to answer queries where data is partioned based on session_id (which is partition key)\
## sorted by item_in_session(cluster key).\
## For quering this table we always require session_id in where clause.

query1 = "CREATE TABLE IF NOT EXISTS song_details"

query1 = query1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")

try:
    session.execute(query1)

except Exception as e:
    print(e)
    
select_query1 = """ SELECT artist, song_title, song_length FROM song_details
                    WHERE session_id = %s 
                    AND item_in_session = %s 
                    """    

                    

In [34]:
# loading Data into table: song_details

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:## INSERT statement

        query1_insert = "INSERT INTO song_details(session_id, item_in_session, artist, song_title, song_length)"
        query1_insert = query1_insert + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [41]:
try:
    rows = session.execute(select_query1,(338,4))
    df1 = pd.DataFrame(rows._current_rows)
except Exception as e:
    print(e)

df1

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [42]:
## query2:
## The following table is designed to  answer queries where data is partioned based on composite  partition key \
## (user_id,session_id) and item_in_session as cluster/sort key.
## For quering this table, we will always require user_id, session_id in where clause \
## and item_in_session(cluster key) is optional.



query2 = "CREATE TABLE IF NOT EXISTS song_details_sorted"

query2 = query2 + (""" (user_id INT,
                        session_id INT,
                        item_in_session INT,
                        artist TEXT,
                        song_title TEXT,
                        user_fname TEXT,
                        user_lname TEXT,
                        PRIMARY KEY ((user_id,session_id), item_in_session)
                                       ) """)

try:
    session.execute(query2)

except Exception as e:
    print(e)



select_query2 = """ SELECT 
                        artist,
                        song_title,
                        item_in_session,
                        user_fname,
                        user_lname
                        
                        FROM song_details_sorted
                        WHERE user_id = %s 
                        AND 
                        session_id = %s 
                    """    




In [43]:
# loading Data into table: song_details_sorted

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:## INSERT statement
        
        insert_query2 = """INSERT INTO song_details_sorted ( user_id
                                                    ,session_id
                                                    ,item_in_session
                                                    ,artist
                                                    ,song_title
                                                    ,user_fname
                                                    ,user_lname
                                                    
                                                    )"""
        
        insert_query2 = insert_query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(insert_query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1],line[4]))                   
                    

In [44]:
# Doing select statement on table: song_details_sorted
try:
    rows = session.execute(select_query2,(10,182))
    df2 = pd.DataFrame(rows._current_rows)
except Exception as e:
    print(e)

df2   

,artist,song_title,item_in_session,user_fname,user_lname
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


In [16]:
## query3:
## The following table is designed to  answer queries where data is partioned based on song_title and
## sorted by user_fname,user_lname
## For quering this table, we will always require song_title in where clause \
## and user_fname , user_lname (cluster key) are optional. If used then they should also be used in order \
## for example user_lname should always come after user_fname in where clause.


query3 = "CREATE TABLE IF NOT EXISTS songs_user"

# Partition key: song_tile , Cluster/Sort keys (user_fname,user_lname)
query3 = query3 + (""" (
                        song_title TEXT,
                        user_fname TEXT,
                        user_lname TEXT,
                        PRIMARY KEY (song_title, user_fname,user_lname)
                                       ) """)

try:
    session.execute(query3)

except Exception as e:
    print(e)



select_query3 = """ SELECT 
                        user_fname,
                        user_lname
                        
                        FROM songs_user
                        WHERE song_title = %s;
                        
                        """
                    

In [17]:
# loading Data into table: songs_user

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        insert_query3 = """INSERT INTO songs_user ( 
                                                    song_title
                                                    ,user_fname
                                                    ,user_lname
                                                    
                                                    )"""
        
        insert_query3 = insert_query3 + "VALUES (%s, %s, %s)"
           
        session.execute(insert_query3, ( line[9],line[1],line[4]))  

In [18]:
# Doing select statement on table: songs_user

try:
    rows = session.execute(select_query3,('All Hands Against His Own', ))
    ##converting query rows into pandas data frame
    df3 =pd.DataFrame(rows._current_rows)
    
except Exception as e:
    print(e)

df3


,user_fname,user_lname
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


In [45]:
## Droping the table 

tables_created = ['song_details','song_details_sorted','songs_user']

#looping through tables_created  list and droping tables

for t in range(len(tables_created)):
    
    try:
        print("Droping table: {}".format(tables_created[t]))
        session.execute("DROP TABLE {}".format(tables_created[t]))
    
    except Exception as e:
        print(e)

    continue


Droping table: song_details
Droping table: song_details_sorted
Droping table: songs_user
Error from server: code=2200 [Invalid query] message="unconfigured table songs_user"


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()